In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Data reading:

In [3]:
os.listdir('/kaggle/input/satellite-image-classification/data/')

In [4]:
print(len(os.listdir('../input/satellite-image-classification/data/cloudy/')))
print(len(os.listdir('../input/satellite-image-classification/data/desert/')))
print(len(os.listdir('../input/satellite-image-classification/data/green_area/')))
print(len(os.listdir('../input/satellite-image-classification/data/water/')))

In [5]:
import matplotlib.pyplot as plt
x=['Cloudy','Desert','Green area','Water']
y=[1500,1131,1500,1500]
plt.bar(x,y);

In [6]:
import tensorflow
from tensorflow import keras
from keras.preprocessing.image import load_img, ImageDataGenerator as IDG

In [7]:
import cv2
load_img('/kaggle/input/satellite-image-classification/data/desert/desert(14).jpg')

### Data Preparation and Visualization:

In [8]:
import tensorflow 
from keras.models import Sequential 
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator as IDG, load_img 
from tensorflow.keras.preprocessing import image_dataset_from_directory

from IPython import display 
display.set_matplotlib_formats('svg')

In [9]:
train_datagen = IDG(rescale=1./255,
                   fill_mode='nearest',
                   featurewise_center=True,
                   validation_split=0.2,
                   horizontal_flip=True,
                   shear_range=0.3,
                   zoom_range=0.25,
                   brightness_range=(0.4,0.7),
                   rotation_range=15,
                   featurewise_std_normalization=True,
                   )


In [10]:
train_generator= train_datagen.flow_from_directory('../input/satellite-image-classification/data',
                                                  target_size=(224,224),
                                                  batch_size=32,
                                                  class_mode='binary',
                                                  subset='training')

In [11]:
test_generator = train_datagen.flow_from_directory('../input/satellite-image-classification/data',
                                                 target_size=(224,224),
                                                 batch_size=32,
                                                 class_mode='binary',
                                                 subset='validation')

In [12]:
train_dataset = image_dataset_from_directory('../input/satellite-image-classification/data',
                                             shuffle=True,
                                             batch_size=32,image_size=(224,224))
class_name =  train_dataset.class_names
plt.figure(figsize = (10,10))
for image , label in train_dataset.take(1):
    for i in range(9) :
        plt.subplot(3,3,i+1)
        plt.imshow(image[i].numpy().astype("uint8"))
        plt.title(class_name[label[i]])
        plt.axis("off")

### Models:

In [13]:
from keras.applications.vgg19 import VGG19
from keras.applications.mobilenet_v2 import MobileNetV2,preprocess_input

In [14]:
vgg_model = VGG19(include_top=True,weights='imagenet')

for layer in vgg_model.layers:
    layer.trainable=False
    

In [15]:
vgg_model = keras.Model(inputs=vgg_model.input,outputs=vgg_model.layers[-2].output)
model= keras.Sequential()

for layer in vgg_model.layers:
    model.add(layer)
    
model.add(Dense(128, activation='relu'))
model.add(Dense(4,activation='softmax'))

In [16]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [18]:
early = tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [19]:
FIT =model.fit(train_generator,
    validation_data = test_generator, 
    callbacks = [early],
    epochs = 50)

In [20]:
model.evaluate(test_generator)